# **Desi System Design -Fine-Tune-SpeechT5**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers==4.38.0 datasets soundfile accelerate==0.28.0 peft==0.10.0 speechbrain==0.5.16

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# prompt: download this dataset and load it "  https://huggingface.co/datasets/keithito/lj_speech

from datasets import load_dataset, Audio
dataset = load_dataset("keithito/lj_speech", split="train")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['id', 'audio', 'file', 'text', 'normalized_text'],
    num_rows: 13100
})

In [5]:
len(dataset)

13100

In [6]:
print(dataset)

Dataset({
    features: ['id', 'audio', 'file', 'text', 'normalized_text'],
    num_rows: 13100
})


In [7]:
# Calculate the number of rows for the full dataset
full_size = len(dataset)

# Select the full dataset
dataset = dataset.select(range(full_size))
print(dataset)


Dataset({
    features: ['id', 'audio', 'file', 'text', 'normalized_text'],
    num_rows: 13100
})


In [8]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [9]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
tokenizer = processor.tokenizer

In [11]:
dataset[2:5]

{'id': ['LJ001-0003', 'LJ001-0004', 'LJ001-0005'],
 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0003.wav',
   'array': array([-0.00404182, -0.00031247, -0.00158895, ...,  0.00022209,
           0.00031626, -0.00021122]),
   'sampling_rate': 16000},
  {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0004.wav',
   'array': array([ 1.84258504e-04, -6.89462904e-05, -2.43774717e-04, ...,
          -5.48143697e-04, -6.62148464e-04, -6.06632268e-04]),
   'sampling_rate': 16000},
  {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0005.wav',
   'array': array([0.00032222, 0.00043797, 0.00043948, ..., 0.00040887, 0.00078317,
          0.        ]),
   'sampling_rate': 16000}

Let's normalize the dataset, create a column called "normalized_text"

In [12]:
def extract_all_chars(batch):
    # Assuming 'text' is the correct column name containing the transcriptions
    all_text = " ".join(batch["text"])  # Changed "transcription" to "text"
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [13]:
dataset_vocab - tokenizer_vocab

{' ',
 '$',
 '&',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '£',
 'à',
 'â',
 'è',
 'ü',
 '’',
 '“',
 '”'}

In [14]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation (except apostrophes)
    text = re.sub(r'[^\w\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['text'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])

{'id': ['LJ001-0003', 'LJ001-0004', 'LJ001-0005'], 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0003.wav', 'array': array([-0.00404182, -0.00031247, -0.00158895, ...,  0.00022209,
        0.00031626, -0.00021122]), 'sampling_rate': 16000}, {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0004.wav', 'array': array([ 1.84258504e-04, -6.89462904e-05, -2.43774717e-04, ...,
       -5.48143697e-04, -6.62148464e-04, -6.06632268e-04]), 'sampling_rate': 16000}, {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0005.wav', 'array': array([0.00032222, 0.00043797, 0.00043948, ..., 0.00040887, 0.00078317,
       0.        ]), 'sampling_rate': 16000}], 'file': ['/root/.cache/huggin

In [15]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [16]:
dataset_vocab - tokenizer_vocab

{' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'à', 'â', 'è', 'ü'}

In [17]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

In [18]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [19]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [20]:
processed_example["speaker_embeddings"].shape

(512,)

In [21]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

In [22]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 300

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

13100

In [23]:
dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)

In [24]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [25]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [26]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [32]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_sysdesign",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=1500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

In [33]:
from accelerate import Accelerator, DataLoaderConfiguration

# Define DataLoaderConfiguration
dataloader_config = DataLoaderConfiguration(
    dispatch_batches=False,  # Each process fetches its own batch
    split_batches=True       # Split fetched batches across processes
)

# Initialize Accelerator with DataLoaderConfiguration
accelerator = Accelerator(dataloader_config=dataloader_config)

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [34]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [35]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,0.468400,0.414132
200,0.449600,0.410773
300,0.433400,0.395516
400,0.424500,0.392135
500,0.422500,0.389238
600,0.420700,0.385765
700,0.415100,0.382024
800,0.413600,0.380311
900,0.410500,0.378225
1000,0.408300,0.376327


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/e

TrainOutput(global_step=1500, training_loss=0.42307407506306965, metrics={'train_runtime': 4769.5561, 'train_samples_per_second': 10.064, 'train_steps_per_second': 0.314, 'total_flos': 5483826441583776.0, 'train_loss': 0.42307407506306965, 'epoch': 4.58})

In [36]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


CommitInfo(commit_url='https://huggingface.co/hijklmno/speecht5_finetuned_sysdesign/commit/7e4ea1616935fef597fd501a91710fe5b8c9c7db', commit_message='End of training', commit_description='', oid='7e4ea1616935fef597fd501a91710fe5b8c9c7db', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hijklmno/speecht5_finetuned_sysdesign', endpoint='https://huggingface.co', repo_type='model', repo_id='hijklmno/speecht5_finetuned_sysdesign'), pr_revision=None, pr_num=None)

TO Test it on hugging face spaces live click Here below:

https://huggingface.co/spaces/Omarrran/tts_model_demo

# Inference

In [37]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "hijklmno/speecht5_finetuned_sysdesign"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [38]:
example = dataset["test"][304 % len(dataset["test"])] # Use modulo operator to wrap around index
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [39]:
text = "As a Data Scientist, I'll be demonstrating my speaking voice in this example. If you don't like my voice, you can choose a different one by setting the speaker parameter."

In [40]:
import re

number_words = {
    0: "zero", 1: "one", 2: "two", 3: "three", 4: "four", 5: "five", 6: "six", 7: "seven", 8: "eight", 9: "nine",
    10: "ten", 11: "eleven", 12: "twelve", 13: "thirteen", 14: "fourteen", 15: "fifteen", 16: "sixteen", 17: "seventeen",
    18: "eighteen", 19: "nineteen", 20: "twenty", 30: "thirty", 40: "forty", 50: "fifty", 60: "sixty", 70: "seventy",
    80: "eighty", 90: "ninety", 100: "hundred", 1000: "thousand"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + ("-" + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        return (number_words[hundreds] + " hundred" if hundreds > 1 else "hundred") + (" and " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        return number_to_words(thousands) + " thousand" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " million" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " billion" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Find the numbers and change with words.
    result = re.sub(r'\b\d+\b', replace, text)

    return result


In [41]:
import re

# Dictionary of technical words and their spoken equivalents (each letter separated)
technical_words = {
    "API": "A P I",
    "CUDA": "C U D A",
    "OAuth": "O Auth",
    "LLM": "L L M",
    "HTTP": "H T T P",
    "HTTPS": "H T T P S",
    "URL": "U R L",
    "SQL": "S Q L",
    "JSON": "J S O N",
    "XML": "X M L",
    "REST": "R E S T",
    "JWT": "J W T",
    "FTP": "F T P",
    "SSH": "S S H",
    "GPU": "G P U",
    "CPU": "C P U",
    "IP": "I P",
    "RAM": "R A M",
    "ROM": "R O M",
    "ID": "I D",
    "UID": "U I D",
    "UUID": "U U I D",
    "NLP": "N L P",
    "ML": "M L",
    "AI": "A I",
    "IoT": "I O T",
    "VPN": "V P N",
    "DNS": "D N S",
    "SMTP": "S M T P",
    "KNN": "K N N",
    "CNN": "C N N",
    "RNN": "R N N",
    "LSTM": "L S T M",
    "GRU": "G R U",
    # Add more technical terms with letters separated
}

def technical_words_to_speech(word):
    # Convert technical words to their spoken form, with each letter separated by spaces
    return technical_words.get(word, word)  # If not in the dictionary, return the original word

def replace_technical_words_with_speech(text):
    # Split the text into words and check for technical terms
    words = re.findall(r'\b\w+\b', text)

    result = []
    for word in words:
        # Convert each word to its spoken form if it's a technical term
        spoken_form = technical_words_to_speech(word)
        result.append(spoken_form)

    return " ".join(result)

# Example usage
text = "I will use an API with OAuth and CUDA to train the LLM model on a GPU."
spoken_text = replace_technical_words_with_speech(text)
print(spoken_text)


I will use an A P I with O Auth and C U D A to train the L L M model on a G P U


In [43]:
inputs = processor(text=spoken_text, return_tensors="pt")

In [45]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

In [46]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'output.wav')
sf.write('output.wav', speech.numpy(), 16000)

In [ ]:
--------------------------------------------------------------------------------------------------------------------------------

TO Test it on hugging face spaces live click Here below:

https://huggingface.co/spaces/Omarrran/tts_model_demo

thank you